# MSCA 31013 BIG DATA PLATFORMS FINAL PROJECT
### GARIMA SOHI

## MODULE - 1 (DATASET SELECTION)

#### 1.1 Importing required packages and libraries for setting up the environment

In [1]:
import os
import shutil
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
import subprocess

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [3]:
directory = 'gs://msca-bdp-tweets/final_project/'
# file = '*.json'
# path = directory + file
path = directory

In [4]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

497.0 G  497.0 G  gs://msca-bdp-tweets/final_project



In [5]:
from google.cloud import storage

In [6]:
import pyspark 
sc = spark.sparkContext
print('Original spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))
conf = pyspark.SparkConf().setAll([('spark.driver.maxResultSize', '16g')])
# here you stop the old spark context with old conf
sc.stop()

Original spark.driver.maxResultSize: 1920m


In [8]:
sc = pyspark.SparkContext(conf=conf)

22/12/05 02:02:20 WARN org.apache.spark.SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69

In [6]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True) 

In [10]:
print('Original spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))

Original spark.driver.maxResultSize: 16g


#### 1.2 Reading the original json file for the project

In [ ]:
df_tweets = spark.read.json(path='gs://msca-bdp-tweets/final_project/')

22/12/05 20:51:08 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 3 for reason Container marked as failed: container_1670262084948_0006_01_000004 on host: hub-msca-bdp-dphub-students-backup-gsohi-sw-9s9q.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/12/05 20:51:08 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 5 for reason Container marked as failed: container_1670262084948_0006_01_000006 on host: hub-msca-bdp-dphub-students-backup-gsohi-sw-9s9q.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/12/05 20:51:08 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 3 on hub-msca-bdp-dphub-students-backup-gsohi-sw-9s9q.c.msca-bdp-students.internal: Container marked as failed: container_1670262084948_0006_01_000004 on host: 

#### 1.3 Converting data format from json to parquet for pre-processing

In [8]:
df_tweets.write.format("parquet").\
mode('overwrite').\
save('gs://' + 'msca-bdp-students-bucket/shared_data/gsohi' + '/bdp_final_project')

22/12/05 21:15:41 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670262084948_0006_01_000001 on host: hub-msca-bdp-dphub-students-backup-gsohi-sw-qkw2.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-05 21:15:41.029]Container killed on request. Exit code is 143
[2022-12-05 21:15:41.030]Container exited with a non-zero exit code 143. 
[2022-12-05 21:15:41.068]Killed by external signal
.
22/12/05 21:15:41 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 1 for reason Container from a bad node: container_1670262084948_0006_01_000001 on host: hub-msca-bdp-dphub-students-backup-gsohi-sw-qkw2.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-05 21:15:41.029]Container killed on request. Exit code is 143
[2022-12-05 21:15:41.030]Container exited with a non-zero exit code 143. 
[2022-12-05 21:15:41.068]Killed by external signal
.
22/12/05 21:15:

In [7]:
df_tweets = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/gsohi/bdp_final_project')

22/12/05 21:41:43 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


#### 1.4 Taking a small random fraction of the big data to understand schema

In [ ]:
twt_sample = df_tweets.sample(fraction=0.0025)

In [9]:
twt_sample.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

As it can be seen from above, file is densely populated with multiple variables. However, I would require only a set of few variables for my project. Let's dive deeper into discovering the required set of variables.

In [10]:
twt_sample.count()

249956

In [11]:
twt_sample.limit(3).toPandas()

,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,...,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_copyright,withheld_in_countries
0,None,Sun Sep 04 20:32:36 +0000 2022,None,"([([14, 27], SundayFunday)], None, [], [(short...",None,None,0,False,low,None,...,MrsDSpEd,"(None, Sun Sep 04 15:25:25 +0000 2022, [0, 140...","<a href=""http://twitter.com/download/android"" ...",RT @MrsDSpEd: #SundayFunday 📍LIST DROP📍\n\nI’m...,1662323556757,False,#SundayFunday 📍LIST DROP📍\n\nI’m a Middle Scho...,"(False, Wed Aug 03 17:06:34 +0000 2022, True, ...",None,None
1,None,Sun Sep 04 20:33:02 +0000 2022,None,"([], None, [], [(twitter.com/LizWest1347657…, ...",None,None,0,False,low,None,...,None,None,"<a href=""http://twitter.com/download/android"" ...","In the school of mortality, we experienc https...",1662323582835,False,"In the school of mortality, we experienc https...","(False, Mon Jul 14 19:40:17 +0000 2014, True, ...",None,None
2,None,Sun Sep 04 20:34:50 +0000 2022,"[27, 140]","([], None, [], [(twitter.com/i/web/status/1…, ...",None,"([27, 222], ([], None, [], [], [Row(id=7085702...",0,False,low,None,...,TheAeronut Awful. Terrible here in the UK. So ...,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@msjanebond007 @TheAeronut Awful. Terrible her...,1662323690839,True,@msjanebond007 @TheAeronut Awful. Terrible her...,"(False, Fri Oct 11 13:08:04 +0000 2013, True, ...",None,None


As we are required to analyze the profiles of Twitterers (not the actual text messages of the Tweets), I am extracting users object columns into spark dataframe to identify relevant columns.

There are deprecated columns like is_translators, profile background, images and urls which we dont need for our analysis. Therefore, I am considering the remaining columns of users to understand how the data looks like.

In [12]:
users = twt_sample.select([
twt_sample.user['created_at'],
twt_sample.user['description'],
twt_sample.user['favourites_count'],
twt_sample.user['followers_count'],
twt_sample.user['friends_count'],
twt_sample.user['id'],
twt_sample.user['listed_count'],
twt_sample.user['location'],
twt_sample.user['name'],
twt_sample.user['protected'],
twt_sample.user['statuses_count'],
twt_sample.user['verified']])

In [13]:
users.dtypes

[('user.created_at', 'string'),
 ('user.description', 'string'),
 ('user.favourites_count', 'bigint'),
 ('user.followers_count', 'bigint'),
 ('user.friends_count', 'bigint'),
 ('user.id', 'bigint'),
 ('user.listed_count', 'bigint'),
 ('user.location', 'string'),
 ('user.name', 'string'),
 ('user.protected', 'boolean'),
 ('user.statuses_count', 'bigint'),
 ('user.verified', 'boolean')]

In [14]:
users.limit(3).toPandas()

,user.created_at,user.description,user.favourites_count,user.followers_count,user.friends_count,user.id,user.listed_count,user.location,user.name,user.protected,user.statuses_count,user.verified
0,Wed Aug 03 17:06:34 +0000 2022,Teacher. Mom. Wife.,972,75,129,1554876248547024897,0,"Michigan, USA",Marie S,False,1382,False
1,Mon Jul 14 19:40:17 +0000 2014,Yayxnmfjkhsan,192,119,484,2711114908,0,Accra,Technics,False,1111,False
2,Fri Oct 11 13:08:04 +0000 2013,watching on in despair as our gov’t destroys us.,33382,4192,4140,709292729,1,None,Trenchy24💙🇪🇺,False,32963,False


As it can be seen in the users data above, User's id, name, description, location, verified and statuses_count will be significant in analysing the profile of twitterers for our project.

Now, identifying the right colum for retweet.

In [16]:
retweets = twt_sample.select([
twt_sample.retweeted_status.user['id'],
twt_sample.retweeted_status.user['name'],
twt_sample.retweeted_status.user['description']  
])

In [17]:
retweets.limit(8).toPandas()

,retweeted_status.user.id,retweeted_status.user.name,retweeted_status.user.description
0,5.863125e+07,Mrs D,Middle School Special Ed ♥️ & Wife Sunny So Ca...
1,NaN,None,None
2,NaN,None,None
3,2.791823e+07,Humble ISD,Ranked #1 BEST Place to Teach in the Houston A...
4,1.322992e+18,Folic Acid😉🇩🇴,I'm Anthony • Pharmacist • How can I lose when...
5,2.457094e+09,PortlandBusinessTrib,"The faces, the stories, the money. Business Tr..."
6,9.801980e+17,NotTheRealTraceyDelaney,"Sometimes I make jokes, sometimes I make serio..."
7,NaN,None,None


For understanding the retweets distribution across twitterers, I will leverage the id, name & description of the users from retweeted_status object.

Further, identifying the right column for geographical view of the tweets by exploring Place object.

In [18]:
geo =  twt_sample.select([
twt_sample.place['country'],
twt_sample.place['country_code'],
twt_sample.place['full_name'],
twt_sample.place['id'],
twt_sample.place['name'],
twt_sample.place['place_type'],
twt_sample.place['url']])

In [19]:
geo.limit(8).toPandas()

,place.country,place.country_code,place.full_name,place.id,place.name,place.place_type,place.url
0,None,None,None,None,None,None,None
1,Kingdom of Saudi Arabia,SA,"Riyadh, Kingdom of Saudi Arabia",01da545b582fc86b,Riyadh,admin,https://api.twitter.com/1.1/geo/id/01da545b582...
2,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None
5,None,None,None,None,None,None,None
6,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None


In [20]:
geo.toPandas().isna().sum()

place.country         247809
place.country_code    247809
place.full_name       247809
place.id              247809
place.name            247809
place.place_type      247809
place.url             247809
dtype: int64

Data is populated for some rows. Thus, place.country can be interesting to explore for country view.

#### 1.5 Creating the final dataset for analysis

Choosing tweets with english language only.

In [21]:
df_tweets_en = df_tweets.filter(df_tweets.lang == 'en')

Selecting all the columns discussed above and creating a new spark dataframe with descriptive labels.

In [22]:
df_tweets_final = df_tweets_en.select([
df_tweets_en.user['id'].alias('user_id'),
df_tweets_en.user['name'].alias('user_name'),
df_tweets_en.user['description'].alias('user_description'),
df_tweets_en.user['location'].alias('user_location'),
df_tweets_en.user['statuses_count'].alias('statuses_count'),
df_tweets_en.user['verified'].alias('user_verified_status'),    
twt_sample.retweeted_status.user['id'].alias('retweet_user_id'),
twt_sample.retweeted_status.user['name'].alias('retweet_user_name'),
twt_sample.retweeted_status.user['description'].alias('retweet_user_description'), 
df_tweets_en.user['followers_count'].alias('user_followers_count'),
df_tweets_en.place['country'].alias('place_country'),   
df_tweets_en.created_at.alias('tweet_created_at'),
df_tweets_en.retweeted.alias('is_retweeted'),
df_tweets_en.tweet_text.alias('text')    
])

In [20]:
df_tweets_final.count()

99992797

In [23]:
df_tweets_final.limit(3).toPandas()

,user_id,user_name,user_description,user_location,statuses_count,user_verified_status,retweet_user_id,retweet_user_name,retweet_user_description,user_followers_count,place_country,tweet_created_at,is_retweeted,text
0,196302166,rach,None,None,13913,False,NaN,None,None,89,None,Sun Sep 04 20:31:13 +0000 2022,,@unicodacorn @Panderoona @Fred11096011 Yes wat...
1,1181979908332896256,Mad Libs Bot,I filled out the Mad Libs so you can jump stra...,None,143268,False,NaN,None,None,8,None,Sun Sep 04 20:31:14 +0000 2022,,HOW TO DATE THE COOLEST GIRL/GUY IN SCHOOL: It...
2,1548987341917806592,sondos,what are youoing here,None,481,False,NaN,None,None,259,None,Sun Sep 04 20:31:14 +0000 2022,,Happy total section for college win capital se...


Discarding irrelevant tweets and choosing tweets with education data by filtering 'text' column with education keywords.

In [24]:
df_tweets_eda = df_tweets_final.filter(
                                     lower(col('text')).contains('primary education') | \
                                     lower(col('text')).contains('secondary education') | \
                                     lower(col('text')).contains('higher education') | \
                                     lower(col('text')).contains('cyberschool') | \
                                     lower(col('text')).contains('home schooling') | \
                                     lower(col('text')).contains('nursery') | \
                                     lower(col('text')).contains('university') | \
                                     lower(col('text')).contains('primary school') | \
                                     lower(col('text')).contains('secondary school') | \
                                     lower(col('text')).contains('postsecondary school') | \
                                     lower(col('text')).contains('junior school') | \
                                     lower(col('text')).contains('high school') | \
                                     lower(col('text')).contains('pre school') | \
                                     lower(col('text')).contains('middle school') | \
                                     lower(col('text')).contains('graduate school') | \
                                     lower(col('text')).contains('graduate schools') | \
                                     lower(col('text')).contains('postgraduate school') | \
                                     lower(col('text')).contains('postgraduate schools') | \
                                     lower(col('text')).contains('elementary school') | \
                                     lower(col('text')).contains('elementary schools') | \
                                     lower(col('text')).contains('public school') | \
                                     lower(col('text')).contains('public schools') | \
                                     lower(col('text')).contains('department of education') | \
                                     lower(col('text')).contains('antischool') | \
                                     lower(col('text')).contains('education department') | \
                                     lower(col('text')).contains('stem education') | \
                                     lower(col('text')).contains('stem degree') | \
                                     lower(col('text')).contains('mba') | \
                                     lower(col('text')).contains('mscs') | \
                                     lower(col('text')).contains('county schools') | \
                                     lower(col('text')).contains('school board') | \
                                     lower(col('text')).contains('charter school') | \
                                     lower(col('text')).contains('private school') | \
                                     lower(col('text')).contains('private schools') | \
                                     lower(col('text')).contains('charter schools') | \
                                     lower(col('text')).contains('private school voucher program') | \
                                     lower(col('text')).contains('student') | \
                                     lower(col('text')).contains('students') | \
                                     lower(col('text')).contains('school voucher') | \
                                     lower(col('text')).contains('text book') | \
                                     lower(col('text')).contains('text books') | \
                                     lower(col('text')).contains('k-12 classroom') | \
                                     lower(col('text')).contains('k-12 classrooms') | \
                                     lower(col('text')).contains('k-12 curriculum') 
                                     )

In [ ]:
df_tweets_eda.count()

37204639

Now, the data is reduced from ~100 Million rows to ~37 Million rows.

Further, in order to reduce the data size, I am selecting tweets having hashtags because hashtags make the user more discoverable and user will have broader reach for the audience.

In [72]:
df_hashtag = df_tweets_eda.filter(F.col('text').like('%#%'))
col_trimmed = F.trim((F.regexp_replace('text', '.*?(#\w+)|.+', '$1 ')))
df_hashtag = df_hashtag.select(F.regexp_replace(col_trimmed,'\s',', ').alias('HashTag'))
df_hashtag.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------+
|HashTag                                                                                                                      |
+-----------------------------------------------------------------------------------------------------------------------------+
|#RamaphosaMustGo                                                                                                             |
|, #AGTG, , , #Family, , , #TTH, , ,                                                                                          |
|#sportscards, #ad                                                                                                            |
|#UnderTheRadarBallerAlert, , , , , , , , , , , , , , , , , , , , , , , , ,                                                   |
|#university, #stammer, , , , , , #support, #stutter, #highereducation, #invisibledisabilities, #hiddend

In [ ]:
list(df_hashtag.select('HashTag').toPandas()['HashTag'])

22/12/04 22:54:14 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670104162554_0035_01_000146 on host: hub-msca-bdp-dphub-students-backup-gsohi-sw-zxqd.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 22:54:14.297]Container killed on request. Exit code is 143
[2022-12-04 22:54:14.298]Container exited with a non-zero exit code 143. 
[2022-12-04 22:54:14.298]Killed by external signal
.
22/12/04 22:54:14 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670104162554_0035_01_000148 on host: hub-msca-bdp-dphub-students-backup-gsohi-sw-zxqd.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 22:54:14.316]Container killed on request. Exit code is 143
[2022-12-04 22:54:14.317]Container exited with a non-zero exit code 143. 
[2022-12-04 22:54:14.329]Killed by external signal
.
22/12/04 22:54:14 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint

['#RamaphosaMustGo',
 ', #AGTG, , , #Family, , , #TTH, , , ',
 '#sportscards, #ad',
 '#UnderTheRadarBallerAlert, , , , , , , , , , , , , , , , , , , , , , , , , ',
 '#university, #stammer, , , , , , #support, #stutter, #highereducation, #invisibledisabilities, #hiddendisabilities, #freshers',
 '#10, , , , , , , ',
 ', , , , , , , , , , , , , , #MobilizeForMidterms',
 ', #ERACED',
 '#Education, #Mooresville',
 '#ASUTwitter',
 '#clearthelist, #clearthelist2022, #teachertwitter, , , , ',
 '#clearthelist, , , , , , , , , , , #AdoptATeacher, #helpateacher, #teachertwitter',
 ', , #NoFlyZoneInTigray, #TigrayGenocide',
 '#parachute, #gradeschool, #PE, #physicaleducation, #teacher, #PreK, #clearthelist2022, #teachertwitter, #DC, #Preschool',
 '#LIAR',
 '#VIDEO',
 ', , , , , , , , , , , #ucuRISING',
 '#clearthelist2022, #clearthelist, #title1, #AdoptATeacher',
 '#ThisIsYale',
 ', , #NoFlyZoneInTigray, #TigrayGenocide',
 '#AmagamaKaNokutela',
 ', #NoFlyZoneInTigray, #TigrayGenocide',
 '#GoWildca

In [26]:
df_tweets_final = df_tweets_eda.filter( col('text').contains('#'))

In [ ]:
df_tweets_final.count()

8350980

With the filter of using hashtags, data size is reduced from ~37 Million to 8 Million.

Saving the file for next module.

In [28]:
df_tweets_final.write.format("parquet").\
mode('overwrite').\
save('gs://' + 'msca-bdp-students-bucket/shared_data/gsohi' + '/bdp_final_project_eda')